# Activity #1: Basic Maps with cartopy

In [1]:
# import our usual things
%matplotlib inline
import cartopy
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets

In [2]:
# lets make our maps a bit bigger for now
plt.rcParams["figure.dpi"] = 300

* this is grabbing a "shape" file for frequently used data
* there are a bunch of specific files you can grab here: https://github.com/nvkelso/natural-earth-vector/tree/master/zips


In [3]:
# ok, lets start thinking about how to link this data to
#  the actual readings of each detector

# first, lets read in the detector data
seismic = pd.read_csv("/Users/jnaiman/Downloads/data_tohoku_norm_transpose.csv",
           header = None)

# lets upload the locations of each of these sensors 
#  during the earthquake
locations = pd.read_csv("/Users/jnaiman/Downloads/location.txt", delimiter="\t",
            header = None, names = ["longitude", "latitude", "empty1", "empty2"])

In [4]:
# we have 3 options: we can decrese the number of stations, 
#  or the number of time samples, or both
# for illustration purposes, lets do both

nstations = 300 # downsample to 300
ntimes = 1440 # factor of 10

import numpy as np
stationsIndex = np.random.choice(range(locations.shape[0]-1), 
                                  nstations, replace=False)
timesIndex = np.random.choice(range(seismic.shape[0]-1), 
                                  ntimes, replace=False)

# sort each
stationsIndex.sort()
timesIndex.sort()

In [5]:
locations2 = locations.loc[stationsIndex]
seismic2 = seismic.loc[timesIndex,stationsIndex]
seismic2.shape, locations2.shape
# sweet

((1440, 300), (300, 4))

In [6]:
# note, we can also do the above plot with bqplot as well:
import bqplot

# scales
x_sc = bqplot.LinearScale()
y_sc = bqplot.LinearScale()

# marks
lines = bqplot.Lines(x = seismic2.index.values, 
                     y = seismic2.iloc[:,0],
                    scales = {'x': x_sc, 'y': y_sc})

# axes
x_ax = bqplot.Axis(scale = x_sc)
y_ax = bqplot.Axis(scale = y_sc, orientation = 'vertical')

# combine into figure
fig = bqplot.Figure(marks = [lines], axes = [x_ax, y_ax])

# create our slider using ipywidgets
slider = ipywidgets.IntSlider(min=0, max=nstations-1)
y_sc.min = -1.0
y_sc.max = 1.0

# create a linking function for slider & plot
def update_slider(event):
    lines.y = seismic2.iloc[:,event['new']]

slider.observe(update_slider, 'value')

display(ipywidgets.VBox([slider, fig]))

# note that this is much more responsive now
#  than we we did this ourselves
# bqplots ftw

In [7]:
# # ok, so we are now super into linking THING A with THING B
# #  so lets link our sesmic data with its location on the map
# #  we can do this with cartopy & matplotlib

# @ipywidgets.interact(station = (0, nstations, 1), 
#                      t = (0, ntimes, 1))
# def plot(station = 0, t = 0):
#     fig = plt.figure(figsize=(10, 10))
#     ax = fig.add_subplot(211, 
#                          projection = cartopy.crs.LambertCylindrical())
#     colors = seismic2.iloc[t]
#     ax.scatter(locations2["longitude"], 
#                locations2["latitude"], 
#                transform = cartopy.crs.PlateCarree(),
#                c = colors)
#     ax.coastlines()

#     ax = fig.add_subplot(212)
#     ax.plot(seismic2.index.values, seismic2.iloc[:,station])
#     ax.set_ylim(-1, 1)

# Activity #3: Info viz maps with bqplot

In [8]:
# with bqplot

map_mark = bqplot.Map(scales={'projection': bqplot.AlbersUSA()})
fig = bqplot.Figure(marks=[map_mark], title='Basic Map Example')
fig

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, marks=[Map(hovered_styles={'hovered_fill…

In [9]:
# can make a statemap instead
#(1)
sc_geo = bqplot.AlbersUSA()
state_data = bqplot.topo_load('map_data/USStatesMap.json')

# (2) 
def_tt = bqplot.Tooltip(fields=['id', 'name'])

states_map = bqplot.Map(map_data=state_data, 
                        scales={'projection':sc_geo}, 
                        tooltip=def_tt)

# (2) grab interactions
states_map.interactions = {'click': 'select', 'hover': 'tooltip'}

# (3) grab data directly from map
# we could also grab from the state_data itself
from states_utils import get_ids_and_names
ids, state_names = get_ids_and_names(states_map)


# lets make into arrays for ease
#state_names =np.array(state_names)
#ids = np.array(ids)
state_names, ids
# into arrays


# (4) data
def get_data_value(change):
    if change['owner'].selected is not None:
        for i,s in enumerate(change['owner'].selected):
            print(state_names[s == ids])
        
states_map.observe(get_data_value,'selected')

# (1)
fig=bqplot.Figure(marks=[states_map], 
                  title='US States Map Example',
                  fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'right': 0}) # try w/o first and see
fig

Figure(fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'right': 0}, marks=[Map(hovered_styles={'hovered_fill': '…

## Adding in some data to link to our usa map

In [10]:
# lets add in some exprot data
comm = pd.read_csv('/Users/jnaiman/Downloads/total_export.csv')

In [11]:
comm.loc[comm['State'] == 'Alabama'].values
# we note that these are formatted as strings - this means we'll have to 
#  do some formatting when we plot data

# also, note that the state name is the first column and not a number
# we'll also have to take care of this too

array([['Alabama', '574.6', '673.2', '530.0', '719.5', '724.4', '771.5',
        '695.6', '839.7', '1,050.4', '1,040.7', '1,131.7', '1,405.0',
        '1,469.6', '1,587.3', '1,510.1', '1,199.1', '1,207.6', '1,271.9']],
      dtype=object)

In [12]:
# grab years
years = list(comm.columns.values)
years = np.array(years[1:]) # get rid of state
# as numbers
years = years.astype('int')
years

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017])

In [13]:
sc_geo = bqplot.AlbersUSA()
state_data = bqplot.topo_load('map_data/USStatesMap.json')

def_tt = bqplot.Tooltip(fields=['id', 'name'])

states_map = bqplot.Map(map_data=state_data, scales={'projection':sc_geo}, tooltip=def_tt)
states_map.interactions = {'click': 'select', 'hover': 'tooltip'}

fig=bqplot.Figure(marks=[states_map], title='US States Map Example',
                  fig_margin={'top': 0, 'bottom': 0, 'left': 0, 'right': 0})


# lets also make a line plot
# second, the lineplot
x_scl = bqplot.LinearScale() 
y_scl = bqplot.LinearScale()
ax_xcl = bqplot.Axis(label='Year', scale=x_scl)
ax_ycl = bqplot.Axis(label='Total Export from State NA', 
                     scale=y_scl, 
                    orientation='vertical', side='left')
lines = bqplot.Lines(x = years, y = np.zeros(len(years)),
                    scales = {'x': x_scl, 'y': y_scl})
#print(lines)
fig_lines = bqplot.Figure(marks = [lines], 
                          axes = [ax_ycl, ax_xcl],)

# let do something additive for all states selected
def get_data_value(change):
    exports = np.zeros(len(years))
    snames = ''
    if change['owner'].selected is not None:
        for i,s in enumerate(change['owner'].selected):
            sn = state_names[s == ids][0]
            snames += sn + ', '
            # because of formatting, things are in arrays hence [0]
            #  also, take out state name hence [1:]
            # NOTE! BQPLOT has misspelled massachussetts!
            if sn == 'Massachusetts': sn = 'Massachussetts'
            exports_in=comm.loc[comm['State'] == sn].values[0][1:]
            # there are ","'s in exports we gotta take out
            exports_in = np.array([exports_in[i].replace(',','') for i in range(len(exports_in))])
            exports = np.add(exports, exports_in.astype('float64'))
        lines.y = exports
        ax_ycl.label='Total Export from ' + snames
    else:
        lines.y = np.zeros(len(exports))
        ax_ycl.label='Total Export from NA'

states_map.observe(get_data_value,'selected')

# some formatting for vertical
#fig_lines.layout.max_height='250px'
#fig_lines.layout.min_width='800px'
#fig.layout.min_width='800px'
#ipywidgets.VBox([fig_lines,fig])
ipywidgets.HBox([fig,fig_lines])

In [14]:
sn = 'Massachusetts'
sn = 'Massachussetts'
print(comm[comm['State'] == sn])


             State   2000  2001   2002   2003   2004   2005   2006   2007  \
20  Massachussetts  106.1  99.1  107.7  111.3  117.6  121.2  142.6  170.3   

     2008   2009   2010   2011   2012   2013   2014   2015   2016   2017  
20  210.0  178.6  197.7  230.6  218.7  206.7  182.4  172.5  164.3  171.4  


In [15]:
comm

State      2000      2001      2002      2003      2004  \
0          Alabama     574.6     673.2     530.0     719.5     724.4   
1           Alaska       6.3       6.9       6.5       8.3       8.4   
2          Arizona     485.9     467.1     511.8     552.6     632.1   
3         Arkansas   1,306.3   1,346.9   1,327.7   1,801.0   1,914.8   
4       California   6,852.6   7,088.0   7,210.3   7,848.4   8,703.1   
5         Colorado     892.6     918.0     918.6     872.5     739.6   
6      Connecticut     132.1     128.6     131.0     130.2     134.7   
7         Delaware     130.7     148.3     122.2     120.7     136.9   
8          Florida   1,883.2   1,857.4   1,797.3   1,882.0   1,877.0   
9          Georgia   1,301.5   1,321.3   1,099.2   1,330.3   1,473.3   
10          Hawaii     151.0     158.0     152.2     159.4     164.1   
11           Idaho     718.1     738.7     769.2     720.0     775.8   
12        Illinois   3,116.2   3,317.8   3,379.4   3,736.9   3,781.8   
13         Indiana   1,710.5   2,014.0   1,804.3   1,851.5   2,229.0   
14            Iowa   3,476.0   3,504.1   3,939.0   4,354.3   4,250.2   
15          Kansas   1,955.0   1,991.1   2,051.7   2,269.7   1,909.2   
16        Kentucky   1,275.4   1,279.1   1,090.9   1,150.1   1,238.5   
17       Louisiana     614.6     606.0     596.3     792.0     799.1   
18           Maine     143.2     143.4     118.7     118.1     120.6   
19        Maryland     338.2     365.2     315.8     279.9     323.8   
20  Massachussetts     106.1      99.1     107.7     111.3     117.6   
21        Michigan     911.4     972.2     964.6   1,104.1   1,138.3   
22       Minnesota   2,207.2   2,210.0   2,517.6   2,844.5   2,811.6   
23     Mississippi     750.1     836.6     776.6   1,137.1   1,156.6   
24        Missouri   1,348.1   1,461.5   1,308.8   1,713.3   1,744.6   
25         Montana     503.6     461.1     502.6     515.9     628.7   
26        Nebraska   2,300.5   2,431.6   2,486.9   2,857.2   2,349.2   
27          Nevada      65.5      69.7      57.2      59.9      55.1   
28   New Hampshire      32.6      33.1      32.0      35.5      37.4   
29      New Jersey     246.5     251.2     251.5     261.8     267.2   
30      New Mexico     322.1     329.5     288.0     316.7     318.2   
31        New York     500.5     512.0     500.0     485.5     541.2   
32  North Carolina   1,839.9   2,020.0   1,721.4   1,832.3   2,159.7   
33    North Dakota   1,232.5   1,356.6   1,622.4   1,686.6   1,718.1   
34            Ohio   1,461.4   1,524.9   1,451.6   1,552.8   1,699.2   
35        Oklahoma     817.7     804.2     823.2     918.5     956.7   
36          Oregon     800.0     855.7     795.8     921.3     997.0   
37    Pennsylvania     697.0     755.5     700.6     727.8     857.5   
38    Rhode Island      12.5      15.6      16.0      17.8      19.0   
39  South Carolina     396.7     432.9     369.9     440.6     501.8   
40    South Dakota   1,211.9   1,217.5   1,199.8   1,393.2   1,326.1   
41       Tennessee     552.7     681.7     630.9     855.7     755.4   
42           Texas   2,695.4   2,911.5   2,809.2   3,262.9   3,566.3   
43            Utah     162.4     174.6     165.6     181.0     173.7   
44         Vermont      48.7      49.3      47.7      51.2      55.4   
45        Virginia     538.5     575.6     530.5     560.6     589.6   
46      Washington   1,313.3   1,405.0   1,411.7   1,557.6   1,650.8   
47   West Virginia      59.3      65.3      55.8      61.3      55.6   
48       Wisconsin     920.7     933.0     990.8   1,088.3   1,127.6   
49         Wyoming     146.6     159.5     135.9     143.1     106.7   
50   United States  51,265.5  53,678.9  53,144.2  59,392.7  61,418.1   

        2005      2006      2007       2008      2009       2010       2011  \
0      771.5     695.6     839.7    1,050.4   1,040.7    1,131.7    1,405.0   
1        9.8       9.5      10.2       10.6      10.3       11.5       12.9   
2      663.5     706.0     843.

In [16]:
state_names

array(['Washington', 'Montana', 'Idaho', 'North Dakota', 'Minnesota',
       'Maine', 'Michigan', 'Wisconsin', 'Oregon', 'South Dakota',
       'New Hampshire', 'Vermont', 'New York', 'Wyoming', 'Iowa',
       'Nebraska', 'Massachusetts', 'Illinois', 'Pennsylvania',
       'Connecticut', 'Rhode Island', 'California', 'Utah', 'Nevada',
       'Ohio', 'Indiana', 'New Jersey', 'Colorado', 'West Virginia',
       'Missouri', 'Kansas', 'Delaware', 'Maryland', 'Virginia',
       'Kentucky', 'Arizona', 'Oklahoma', 'New Mexico', 'Tennessee',
       'North Carolina', 'Texas', 'Arkansas', 'South Carolina', 'Alabama',
       'Georgia', 'Mississippi', 'Louisiana', 'Florida', 'Hawaii',
       'Alaska'], dtype='<U14')

In [17]:
comm['State'].index

RangeIndex(start=0, stop=51, step=1)

In [18]:
import pandas as pd
buildings = pd.read_csv("/Users/jnaiman/Downloads/building_inventory.csv",
                        na_values = {'Year Acquired': 0, 'Year Constructed': 0, 'Square Footage': 0})

In [19]:
import numpy as np
nsamples =100
dsm = np.random.choice(range(len(buildings)-1),nsamples,replace=False)

In [20]:
dsm

array([8089, 2233, 4861, 1667,  916, 2489, 8294, 4818, 1174, 3284, 6034,
        134, 6463, 3684, 6318, 2313, 8086, 2315, 6090, 3334,  390, 4618,
       7840, 6707, 2433, 6793, 1952, 2127, 4221, 3790, 8345, 1860, 3057,
       6279, 6687, 7677, 5392, 4162, 7826, 5945, 2615, 3320,   77, 3999,
       6384, 5985, 5843, 3985, 2873, 4806, 8676, 2445,  895, 3943, 7392,
       6575, 1870, 5374, 5713, 5265,  377, 4306, 6383, 1697, 3281,  439,
       7411,  165, 5240, 5479, 2351,  688, 4539, 1242,  426, 3857, 1524,
       2876, 8538, 7043, 1776, 8254, 2236, 3760, 5999, 3989, 3899, 6732,
       7450, 3326, 3011, 8847, 3973, 5640, 3337,  696, 8559, 6314, 3704,
       3493])

In [21]:
buildingsDS = buildings.loc[dsm]

In [22]:
len(buildingsDS)

100

In [23]:
import bqplot
x_scl = bqplot.LinearScale()
y_scl = bqplot.LinearScale()
cd = buildings['Congress Dist']
an = buildings['Agency Name']
sf = buildings['Square Footage']

i,j = 0,0
cdNames = cd.unique()
anNames = an.unique()

mask = (cd.values == cdNames[i]) & (an.values == anNames[j])
ya = buildings['Year Acquired'][mask]
yaNames = ya.unique()
sfNames2 = [sf[mask][ya == yaNames[b]].sum() for b in range(len(yaNames)) ] 
sfNames2 = np.array(sfNames2) 
yfLine = bqplot.Lines(x=yaNames, 
                      y=sfNames2,
                      colors=['Blue'],
                      scales={'x': x_scl, 'y': y_scl}) 
fig = bqplot.Figure(marks=[yfLine])
fig

Figure(fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, marks=[Lines(colors=['Blue'], interactio…